## Imports

In [1]:
import copy
import os
import random
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical

# Lab 6: Project CERN

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Download, cache and extract bird song data

In [3]:
dataset_dir = Path('datasets')
remote_dataset_file='/content/drive/My Drive/dataset_v2.tar.gz'
#data_for_processing = keras.utils.get_file(myFile, 'file://'+fullPath)
if not (dataset_dir/'testing_list.txt').exists(): # Assume dataset already downloaded/extracted if testing list is present
    get_file(None, "file://" + remote_dataset_file,
                    extract=True,
                    cache_dir='.',
                    cache_subdir=dataset_dir)

## Load Bird song data

In [4]:
# Classes to handle, ordered by label
CLASSES = ['alauda_arvensis', 'cuculus_canorus', 'delichon_urbicum', 'emberiza_cirlus', 'emberiza_citrinella', 'falco_tinnunculus', 'muscicapa_striata', 'sylvia_borin', 'tachybaptus_ruficollis', 'tyto_alba']

with (dataset_dir/'testing_list.txt').open() as f:
    testing_list = f.read().splitlines()

x_train = []
y_train = []
x_test = []
y_test = []

for recording in dataset_dir.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        print("avoid")
        continue
    label = CLASSES.index(recording.parent.name) # Assign class number
    
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer
        
    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((16000, 1)) # Resize to 1s (16kHz) with zero-padding, 1 channel

    if str(recording.relative_to(dataset_dir)) in testing_list: # Assign to test set if file in test list
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

## Normalize data

In [5]:
# Use train set as a reference for mean/std
x_mean = x_train.mean()
x_std = x_train.std()

x_train -= x_mean
x_test  -= x_mean
x_train /= x_std
x_test  /= x_std

## Export small dataset (250 random vectors)

In [6]:
perms = np.random.permutation(len(y_test))[0:250]
x_test_250 = x_test[perms]
y_test_250 = y_test[perms]
np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model M5

In [7]:
model = Sequential()
model.add(Input(shape=(16000, 1)))

model.add(MaxPool1D(pool_size=10))
model.add(Conv1D(filters=32, kernel_size=80, strides=4, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=3))

model.add(Flatten())
model.add(Dense(units=10))
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=10e-4)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d (MaxPooling1D  (None, 1600, 1)          0         
 )                                                               
                                                                 
 conv1d (Conv1D)             (None, 381, 32)           2592      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 95, 32)           0         
 1D)                                                             
                                                                 
 conv1d_1 (Conv1D)           (None, 93, 32)            3104      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 23, 32)           0         
 1D)                                                             
                                                        

## Train model

In [8]:
model.fit(x_train, y_train, epochs=10, batch_size=150, validation_data=(x_test, y_test))

Epoch 1/10
155/155 [==============================] - 8s 24ms/step - loss: 2.1560 - categorical_accuracy: 0.2475 - val_loss: 1.9996 - val_categorical_accuracy: 0.3441
Epoch 2/10
155/155 [==============================] - 4s 26ms/step - loss: 1.8617 - categorical_accuracy: 0.3891 - val_loss: 1.8987 - val_categorical_accuracy: 0.3660
Epoch 3/10
155/155 [==============================] - 3s 21ms/step - loss: 1.6804 - categorical_accuracy: 0.4430 - val_loss: 1.6045 - val_categorical_accuracy: 0.4774
Epoch 4/10
155/155 [==============================] - 3s 16ms/step - loss: 1.5635 - categorical_accuracy: 0.4818 - val_loss: 1.5394 - val_categorical_accuracy: 0.4931
Epoch 5/10
155/155 [==============================] - 2s 14ms/step - loss: 1.4729 - categorical_accuracy: 0.5162 - val_loss: 1.4413 - val_categorical_accuracy: 0.5378
Epoch 6/10
155/155 [==============================] - 2s 13ms/step - loss: 1.4083 - categorical_accuracy: 0.5402 - val_loss: 1.4111 - val_categorical_accuracy: 0.556

## Evaluate model on test dataset

In [9]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

312/312 - 1s - loss: 1.2873 - categorical_accuracy: 0.5894 - 1s/epoch - 3ms/step
312/312 [==============================] - 1s 2ms/step
tf.Tensor(
[[476  12  78  41  44  27  35 150  66  31]
 [  3 715  74  18  10  46  58  13  38  19]
 [  6  36 588  26   9  75 195  24  37  27]
 [  9   9 109 458  41  80  84  51 105  32]
 [ 18  15  55  34 388 173  87  98  64  47]
 [  1  20  54  13  30 642  67  57  71  63]
 [  3  31  97  22  29 162 451  37  70  66]
 [ 23   3  35  11  45 111  20 787  30   5]
 [  5  14  42  37   9 141  88  16 567  38]
 [  4  15  32  26   8  32  71   0  29 803]], shape=(10, 10), dtype=int32)


## Evaluate model on small dataset

In [10]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 1.2698 - categorical_accuracy: 0.5760 - 139ms/epoch - 17ms/step
8/8 [==============================] - 0s 3ms/step
tf.Tensor(
[[ 9  0  3  0  2  1  1  4  2  2]
 [ 0 11  0  0  1  1  1  0  0  0]
 [ 0  1 17  0  0  3  6  0  0  2]
 [ 0  0  3  8  1  3  4  1  2  0]
 [ 0  0  1  1 14  4  3  2  4  1]
 [ 0  0  2  0  1 15  0  0  1  1]
 [ 0  2  1  2  0  6 18  0  2  2]
 [ 1  0  0  0  3  2  1 17  0  0]
 [ 1  0  2  1  0  5  2  1 14  1]
 [ 0  0  0  0  0  0  4  0  2 21]], shape=(10, 10), dtype=int32)


## Save trained model

In [11]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [12]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [13]:
!pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Generate C code for the trained model with 16-bit fixed-point representation

In [14]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

———————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                           | Layer                            | Outputs                         
———————————————————————————————————————————————————————————————————————————————————————————————————————
                                 | input_1                          | max_pooling1d                   
-------------------------------------------------------------------------------------------------------
input_1                          | max_pooling1d                    | conv1d                          
-------------------------------------------------------------------------------------------------------
max_pooling1d                    | conv1d                           | max_pooling1d_1                 
-------------------------------------------------------------------------------------------------------
conv1d                           | max_pooling1d_1                  

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [17]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv

gsc_output_fixed/model.c: In function ‘void cnn(const number_t (*)[16000], number_t*)’:
gsc_output_fixed/model.c:147:18: warning: left operand of comma operator has no effect [-Wunused-value]
  147 |     activations1.max_pooling1d_4_output,
      |     ~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~
Testing accuracy: 0.58
